In [2]:
import re
import json
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import AlignIO
from Bio.SeqFeature import SeqFeature, FeatureLocation

In [25]:
def write_gene_fastas(subtype, gene):
    
    #find location of genes
    reference = '../config/rsv_reference.gb'
    for record in SeqIO.parse(open(reference,"r"), "genbank"):
        for feature in record.features:
            if feature.type == 'CDS':
                if feature.qualifiers['gene'] == [gene]:
                    gene_location = feature.location
    
    
    
    
    # alignment file only has accession number associated with a sequence
    # need full id (including location and date, etc)
    # get this from original data/rsv_X.fasta file
    accession_to_id = {}
    
    data_file = f'../data/rsv_{subtype}_genome.fasta'
    with open(data_file, 'r') as handle:
        for virus in SeqIO.parse(handle, 'fasta'):
            accession_to_id[virus.id.split('|')[0]] = virus.id

    
    
    # read in alignment file to find sequence of the specified gene    
    aligned_file = f'../results/aligned_{subtype}.fasta'
    
    # for each virus in the alignment file, save new entry with only the sequence of the specified gene
    gene_records = []

            
    with open(aligned_file, 'r') as handle:

        for virus in SeqIO.parse(handle, 'fasta'):
            gene_seq = gene_location.extract(virus.seq)
            
            gene_records.append(SeqRecord(gene_seq, id=accession_to_id[virus.id], 
                                          description=accession_to_id[virus.id]))
            
    SeqIO.write(gene_records, f'../data/rsv_{subtype}_{gene}.fasta', "fasta")
            
            
            

In [27]:
subtypes = ['A', 'B', 'all']
genes = ['F', 'G']

for s in subtypes:
    for g in genes:
        write_gene_fastas(s, g)